In [1]:
import os
import sys
import nltk
import csv
import re
import pandas as pd
import numpy as np

/usr/local/lib/python3.6/site-packages/sklearn/utils/fixes.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
cluster_data = pd.read_csv('post_data/tweets_clustered.csv')
cluster_dict = dict(zip(cluster_data.id.apply(str), cluster_data.cluster))
user_dict = dict(zip(cluster_data.id.apply(str), cluster_data.name))

In [3]:
clusters = pd.read_csv('post_data/cluster_locations.csv')

cluster_numbers = sorted(set(cluster_data.cluster))


In [4]:
## create corpus

In [5]:
# read posts from file

posts = {x: [] for x in cluster_numbers}
posts_by_user = {x: {} for x in cluster_numbers}
stopWords = set(nltk.corpus.stopwords.words('english'))
stopWords.update(['love', 'day', 'like', 'good', 'time', 'today', 'night',
                  'got', 'great', 'one', 'tonight', 'get'])

with open('CSVs/instagram_data_20170825-155658.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile)

    for row in spamreader:
        cluster = cluster_dict[row[0]]
        username = user_dict[row[0]]
        
        untagPost = re.sub('@[\w.-]+', '', row[2]).lower()
        #words = []
        for word in nltk.word_tokenize(untagPost):
            if word not in stopWords and re.match('^[A-Za-z]+$', word) and len(word) > 2:
                #words.append(word)
                if username in posts_by_user[cluster]:
                    posts_by_user[cluster][username].append(word)
                else:
                    posts_by_user[cluster][username] = [word]
        #text = ' '.join(words)
        #posts[cluster].append(text)

for x in cluster_numbers:
    locposts = posts_by_user[x]
    for user in locposts.keys():
        words = set(locposts[user])
        text = ' '.join(words)
        posts[x].append(text)

    

In [6]:
# # write to files
# corpusdir = os.path.join('post_data', 'corpus')
# corpus_files = []
# for clustnum in posts.keys():
#     outfile = os.path.join(corpusdir,
#                            'instagram_cluster_' + str(clustnum) + '.txt')
    
#     with open(outfile, 'w') as f:
#         for post in posts[cluster]:
#             f.write(post)
#     corpus_files.append(outfile)


In [7]:
vocab = set([l.lower() for l in nltk.corpus.words.words()])

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
tfidf = TfidfVectorizer(min_df = 5, vocabulary = vocab)


post_text_tfidf = [' '.join(posts[p]) for p in cluster_numbers]

tfs = tfidf.fit_transform(post_text_tfidf)

features = tfidf.get_feature_names()

dense = tfs.todense()



In [9]:
# basing code off of http://www.markhneedham.com/blog/2015/02/15/pythonscikit-learn-calculating-tfidf-on-how-i-met-your-mother-transcripts/

topwords = {}
for x in range(0, len(dense)):
    citywords = []
    cluster = cluster_numbers[x]
    row = dense[x].tolist()[0]
    phrase_scores = [pair for pair in zip(range(0, len(row)), row) if pair[1] > 0]
    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
    for phrase, score in [(features[word_id], score) for (word_id, score) in sorted_phrase_scores][:10]:
        citywords.append(phrase)
    topwords[cluster] = citywords
        

In [51]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout = 10)

addressnames = []
for c in cluster_numbers:
    if c >= 0:
        lat = clusters.iloc[c]['lat']
        long = clusters.iloc[c]['long']
        coordstr = str(lat) + ', ' + str(long)
        location = geolocator.reverse(coordstr)
        address = location.raw['address']
        
        addelem = []
        if 'city' in address:
            addelem.append(address['city'])
        elif 'town' in address:
            addelem.append(address['town'])
        elif 'village' in address:
            addelem.append(address['village'])
        elif 'locality' in address:
            addelem.append(address['locality'])
        elif 'suburb' in address:
            addelem.append(address['suburb'])
                            
        if 'state' in address:
            addelem.append(address['state'])
        if 'country' in address:
            addelem.append(address['country'])
        
        addressnames.append(', '.join(addelem))


In [52]:

cnums = [c for c in cluster_numbers if c >= 0]

topwords_by_cluster = pd.DataFrame({ 'clustnum' : cnums,
             'lat' : [clusters.iloc[x]['lat'] for x in cnums],
             'long' : [clusters.iloc[x]['long'] for x in cnums],
             'words' : [' '.join(topwords[x]) for x in cnums],
             'address' : addressnames})
topwords_by_cluster.to_csv('post_data/top_words.csv', index = False, sep = ';')
